In [1]:
# -*- coding:utf-8 -*-
import sys
import csv
import gzip
import json
import math
import datetime
import gc

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle

import cv2
from PIL import Image

import os

#import tdclient
#import pandas_td as td
 
import google.cloud.storage
from google.cloud import storage

import re

import shutil

import tempfile

from flask import escape
import time
from datetime import datetime
import pytz



os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/tani_kyuichiro/nurve-cloud-98b3f-bb7c97f8fb03.json'

#%matplotlib inline


In [60]:
#メソッド


#-------------------------------- メソッド -------------------------------------

def json_paser(json):
    json_open = open(json, 'r')
    json_load = json.load(json_open)

    print(json_load)

#json用座標補正メソッド
def corrected_data_json(line):
    xy_lst = []
    #　小数点以下切り上げ
    #line[1] = math.ceil(float(line[1]))
    #line[2] = math.ceil(float(line[2]))
    x = math.ceil(float(line[0]))
    y = math.ceil(float(line[1]))
    
    #x軸補正
    #if line[1] < 0:
    if x < 0:    
        #xy_lst.append(line[1]*(-1) + 90)
        xy_lst.append(x*(-1) + 90)
    else:
        #xy_lst.append((line[1] - 90)*(-1))
        xy_lst.append((x - 90)*(-1))
     #補正無しX軸   
    #xy_lst.append(line[1])
    
    #y軸補正
    #fix_y = line[2] + 180
    fix_y = y + 180
    if fix_y > 360 :
        xy_lst.append(fix_y - 360)
    else:
        xy_lst.append(fix_y)
    return xy_lst


#余白削除メソッド
def margin_cut(file_name='./hm.jpg',output_name='./hm_edge.png'):
    filename = file_name #'./image_3453421.jpg'
    img = cv2.imread(filename)
    img2 = cv2.imread(filename)

    # Grayscale に変換
    img_gs = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2 値化
    ret, thresh = cv2.threshold(img_gs, 250, 255, cv2.THRESH_BINARY)

    # 輪郭を取得
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # i = 1 は画像全体の外枠になるのでカウントに入れない
    x1 = []
    y1 = []
    x2 = []
    y2 = []
    for i in range(1, len(contours)):
        # ret の中身は (x, y, w, h)
        ret = cv2.boundingRect(contours[i])
        x1.append(ret[0])
        y1.append(ret[1])
        x2.append(ret[0] + ret[2])
        y2.append(ret[1] + ret[3])

    x1_min = min(x1)
    y1_min = min(y1)
    x2_max = max(x2)
    y2_max = max(y2)

    # 枠取りをした結果を表示
    #cv2.rectangle(img, (x1_min, y1_min), (x2_max, y2_max), (0, 255, 0), 2)
    #cv2.imwrite('cropped_edge_rectangle.jpg', img)

    # ギリギリで切り出し
    crop_img = img2[y1_min:y2_max, x1_min:x2_max]
    cv2.imwrite(output_name, crop_img)

    #画像メモリ開放     # 2020/5/13追加
    del crop_img
    gc.collect()

#GCSからの画像データダウンロード
def download_blob(bucket_name, source_blob_name, destination_file_name):

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    """
    print(
        "Blob {} downloaded to {}.".format(
            source_blob_name, destination_file_name
        )
    ) 
    """


#データ量計測
def data_len(*lst):
    lines = list(lst) #list形式に変換一次
    #print(type(lines))
    line_s = lines[0] #list形式に変換二次
    return len(line_s)
        

#ヒートマップデータ作成
def data_maker(*axis_data_lst):
    lst_2d = [] #ヒートマップデータ配列
    x_ranges = [] #x軸bin範囲配列
    x_cnt = [] #x軸bin配列

    #x軸範囲設定
    i = 0
    for x in range(0,360,18):
        x_cnt.append(i)
        x_ranges.append(x)
        i += 1
    
    #x軸データ管理配列設定   
    x_length = len(x_cnt)    

    #x:yデータ集計
    for i in range(0,180,10):
        x_lst = [0] * x_length #x_lstを0初期化
        for xy in  axis_data_lst:
            x = xy[0]
            if x >= i and x < i + 10:
                for cnt, x_range in zip(x_cnt, x_ranges):
                    if xy[1] >= x_range and xy[1] < x_range + 18:
                            x_lst[cnt] += 1
        lst_2d.append(x_lst)
    print('lst_2d: {}'.format(lst_2d))
    return lst_2d

#v_max設定
def v_max_set(*lst_2d):
    x_min = 1000
    x_max = 0
    for xs in lst_2d:
        """
        for x in xs:
            if x_max < x:
                x_max = x
            if x_min > x:
                x_min = x
        """
        x_max = max(xs)
        x_min = min(xs)

    # vのレンジ幅の設定
    if x_max >= 500:
        v_max = 350
    elif x_max >= 150:
        v_max = 100
    elif x_max >= 100:
        v_max = 100
    elif x_max >= 50:
        v_max = 50
    elif x_max >= 25:
        v_max = 15
    elif x_max >= 5:
        v_max = 5
    else:
        v_max = x_max
    return v_max

#CDSバケット内リスト取得、拡張子抽出
def extension_get(bucket_name,prefix):
    storage_client = storage.Client()

        # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(
            bucket_name, prefix=prefix, delimiter='/'
        )

    blob_lst = []
    target = prefix+'2048x1024.'

    for blob in blobs:
        blob_lst.append(blob.name)
    l_in = [s for s in blob_lst if target in s] 

    if len(l_in) > 0:
        extension = l_in[0].replace(target,'')
        #print('ext: {}'.format(extension))
        return extension
    else:
        return False

#バケット内のオブジェクトリスト取得    
def list_blobs(bucket_name):
        """Lists all the blobs in the bucket."""
        # bucket_name = "your-bucket-name"

        storage_client = storage.Client()

        # Note: Client.list_blobs requires at least package version 1.17.0.
            
        bucket = storage_client.get_bucket('rent-production')
        blobs = bucket.list_blobs(prefix="medium_items/media/", delimiter="/")
        dirs = []
        for page in blobs.pages:
            dirs.extend(page.prefixes)

        [ print(x) for x in dirs ]

#オリジナルヒートマップ作成
def heapmap_org_maker(dir_name,*lst_2d):
    # vのレンジ幅の調整
    v_max = v_max_set(*lst_2d)
    #print('vmax:{}'.format(v_max))


    #凡例付きヒートマップ作成
    hm_name_sample = dir_name + "/hm_org.jpg"
    plt.figure(figsize=(40,16)) 
    #fig, ax = plt.subplots(figsize=(32,16))
    plt.tick_params(labelsize=30)
    sns.heatmap(lst_2d,vmin=0, vmax=v_max)
    #sns.heatmap(lst_2d,vmin=0, vmax=5)
    plt.savefig(hm_name_sample )
    #print('凡例付きヒートマップ作成完了')

    # 2020/5/13追加
    plt.close()
    gc.collect()

    #合成用ヒートマップ作成
    hm_name = dir_name + "/hm.png" #<- dirをつける
    plt.figure(figsize=(32,16)) 
    sns.heatmap(lst_2d,vmin=0, vmax=v_max,yticklabels=False,xticklabels=False,cbar=False)
    plt.savefig(hm_name)
    #print('合成用ヒートマップ作成完了')
    
    # 2020/5/13追加
    plt.close()
    gc.collect()

    #HM余白削除
    outfile = dir_name + '/hm_edge.png'
    #outfile = margin_cut(hm_name,outfile)
    margin_cut(hm_name,outfile)
    #print('余白削除処理完了')

#元画像取得
def org_img_get(dir_name,media_id):
    bucketname = 'rent-production'
    prefix = 'medium_items/media/'+media_id + '/'
    ext = extension_get(bucketname,prefix) #拡張子吸い出し
    if not ext:
        print('拡張子なし')
        return False
    
    #bucket_name = 'rent-production/medium_items/media/'+media_id
    bucket_name = 'rent-production'
    source_blob_name = 'medium_items/media/'+media_id + '/2048x1024.' + ext
    file_name = dir_name + '/2048x1024.jpg'
    try:
        download_blob(bucket_name,source_blob_name,file_name)
    except:
        print('file not found. {}'.format(source_blob_name))
        return False
        
    #print('オリジナル画像読み込み完了')

#画像合成
def img_maker(dir_name):
    #画像データ読み込み
    filename = dir_name + '/2048x1024.jpg'
    try:
        img = cv2.imread(filename)
        height = img.shape[0] # Errorを引っ掛けるためだけの仕掛
    except  AttributeError:
        print('file not found. {}'.format(filename))
        return False

    #オリジナル画像のリサイズ
    height = img.shape[0]
    width = img.shape[1]
    if height != 1024 or width != 2048:
        img = cv2.resize(img , (int(width*(2048.5/width)), int(height*(1024.5/height))))
    #--- ここまでオミット対象 ---
    

    #余白除去後HM読み込み
    filename = dir_name + '/hm_edge.png' # <- dirをつける
    try: 
        img2 = cv2.imread(filename,-1)
        height = img2.shape[0] # Errorを引っ掛けるためだけの仕掛
    except  AttributeError:
        print('file not found.')

    #サイズ調整
    height = img2.shape[0]
    width = img2.shape[1]

    img2_1 = cv2.resize(img2 , (int(width*(2048/width)), int(height*(1024/height))))
    #print('orgサイズ:{}'.format(img.shape))
    #print('リサイズhm:{}'.format(img2_1.shape))
    cv2.imwrite(dir_name + '/resize_heatmap.png', img2_1)
    #print('ヒートマップのサイズ調整完了')

    
    #--- ここから機能オミット ---
    #画像合成
    blended = cv2.addWeighted(src1=img,alpha=0.6,src2=img2_1,beta=0.4,gamma=0.3)

    #作成日時刻印 ---------------------------------------------------------------------------------------------
    str_time = datetime.now(pytz.timezone('Asia/Tokyo')).strftime("%Y-%m-%dT%H:%M:%S")
    cv2.putText(blended, str_time, (10, 30),cv2.FONT_HERSHEY_PLAIN, 1.5,(255, 255, 255), 1, 8)


    #合成後画像保存
    cv2.imwrite(dir_name + '/blended_test.jpg', blended) # <- dirをつける
    #print('合成画像作成完了')

    # 2020/5/13追加
    del img2
    del img2_1
    del blended
    gc.collect()
    #--- ここまで機能オミット ---
    
    """
    #半透明画像作成
    im_rgb = Image.open(dir_name + '/resize_heatmap.png')
    im_rgb.putalpha(128)
    im_rgb.save(dir_name + '/pillow_putalpha_solid.png')
    """

    #凡例切り出し準備
    im = Image.open(dir_name + "/hm_org.jpg")

    #凡例切り出し&resize
    #im_crop = im.crop((2230,120,2350,1030))
    im_crop = im.crop((3150,170,3250,1450)) #2020/04/16 凡例切り出しのズレが生じていたので修正
    im_crop_rsize = im_crop.resize((100,1024))   
    #plt.imshow(im_crop_rsize)
    im_crop_rsize.save(dir_name + '/colorbar_crop.jpg', quality=100) # <- dirをつける
    #print('凡例作成完了')

    # 2020/5/13追加
    del im
    del im_crop
    del im_crop_rsize
    gc.collect()


#画像合成
def img_blend(dir_name):
    #--- ここから機能オミット ---
    #ブランク画像作成
    height = 1024
    width = 2200
    blank = np.zeros((height, width, 3))
    blank += 255 #←全ゼロデータに255を足してホワイトにする
 
    cv2.imwrite(dir_name + '/blank.jpg',blank) # <- dirをつける
    #print('ベース画像作成完了')


    #凡例付きヒートマップ作成
    img0 = cv2.imread(dir_name + '/blank.jpg') # ブランク画像
    img1 = cv2.imread(dir_name + '/blended_test.jpg') # ヒートマップ合成画像
    img2 = cv2.imread(dir_name + '/colorbar_crop.jpg') # 凡例

    img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    base_img = img0


    #ヒートマップ画像貼り付け基準点設定
    x_offset=0
    y_offset=0

    #ヒートマップ画像合成
    base_img[y_offset:y_offset+img1.shape[0], x_offset:x_offset+img1.shape[1]] = img1

    #凡例画像貼り付け基準点設定
    x_offset=2050
    y_offset=0

    #凡例画像合成
    base_img[y_offset:y_offset+img2.shape[0], x_offset:x_offset+img2.shape[1]] = img2
    base_img = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGB)

    #media_dir = './medium_items/media/' + media_id 
    media_dir = dir_name + '/' + media_id + '.jpg' # 凡例付き合成画像

    #if not os.path.exists(media_dir):
        #os.makedirs(media_dir) #dirが無い場合作成

    #dir_file_name = './medium_items/media/' + media_id + '/heatmap.jpg'

    #cv2.imwrite(dir_file_name,base_img)
    cv2.imwrite(media_dir,base_img)
    #print('凡例付き合成画像作成完了')
    #--- ここまで機能オミット ---
    
    #--- ここから機能オミット ---
    #合成ヒートマップ画像アップローダー
    #- 本番環境設定 -
    #bucket_name = 'rent-production'
    #destination_blob_name = 'reports/heat_maps/hm/media/' + media_id + '/heatmap.jpg' 
 

#jsonで処理するタイプ    
def heatmap_maker_json(file,dir_name):
    #-----------------------------------  json処理  -------------------------------------------------
    #print('read json.')
    df_s = pd.read_json(file)


    #読み込み画像データ設定
    #print('check media_id.')
    num = re.search(r'[\d]+\.json$',file)
    media_id = num.group().replace('.json','')

    #jsonファイル削除     # 2020/5/13追加
    os.remove(file)

    #item_id =  df_s.loc[0]['item_id']
    #organization_group_id =  df_s.loc[0]['organization_group_id']
    lines =  df_s['coordinate'].values.tolist()
    
    #df削除     # 2020/5/13追加
    del df_s
    gc.collect()
    
    """#読み込み画像データ設定
    line = lines[0]
    file = str(int(line[1])) #item_id
    media_id = str(int(line[0])) #media_id"""

    #print('media_id:{}'.format(media_id))

    #------------------------------ データ作成 -------------------------------------
    #座標補正処理 Ver02
    axis_data_lst = []
    f_lst = []
    idx = 0
    for line in lines:    
        data = corrected_data_json(line) #座標補正
        axis_data_lst.append(data)
    #print('座標補正完了')

    #ヒートマップデータ作成Ver02
    lst_2d = data_maker(*axis_data_lst)
    #print('ヒートマップデータ作成完了')

    #---------------------------------- ヒートマップ作成 ---------------------------------------------
    heapmap_org_maker(dir_name,*lst_2d)
    """
    # vのレンジ幅の調整
    v_max = v_max_set(*lst_2d)
    #print('vmax:{}'.format(v_max))


    #凡例付きヒートマップ作成
    hm_name_sample = dir_name + "/hm_org.jpg"
    plt.figure(figsize=(40,16)) 
    #fig, ax = plt.subplots(figsize=(32,16))
    plt.tick_params(labelsize=30)
    sns.heatmap(lst_2d,vmin=0, vmax=v_max)
    #sns.heatmap(lst_2d,vmin=0, vmax=5)
    plt.savefig(hm_name_sample )
    #print('凡例付きヒートマップ作成完了')

    # 2020/5/13追加
    plt.close()
    gc.collect()

    #合成用ヒートマップ作成
    hm_name = dir_name + "/hm.png" #<- dirをつける
    plt.figure(figsize=(32,16)) 
    sns.heatmap(lst_2d,vmin=0, vmax=v_max,yticklabels=False,xticklabels=False,cbar=False)
    plt.savefig(hm_name)
    #print('合成用ヒートマップ作成完了')
    
    # 2020/5/13追加
    plt.close()
    gc.collect()

    #HM余白削除
    outfile = dir_name + '/hm_edge.png'
    #outfile = margin_cut(hm_name,outfile)
    margin_cut(hm_name,outfile)
    #print('余白削除処理完了')
    """
    
    #--- ここからオミット対象 ---
    #------------------------------------- GCSからの画像データ読み込み -------------------------------------------------
    #直接画像読み込み
    #ファイルの存在確認、拡張子取得
    org_img_get(dir_name,media_id)
    """
    bucketname = 'rent-production'
    prefix = 'medium_items/media/'+media_id + '/'
    ext = extension_get(bucketname,prefix) #拡張子吸い出し
    if not ext:
        print('拡張子なし')
        return False
    
    #bucket_name = 'rent-production/medium_items/media/'+media_id
    bucket_name = 'rent-production'
    source_blob_name = 'medium_items/media/'+media_id + '/2048x1024.' + ext
    file_name = dir_name + '/2048x1024.jpg'
    try:
        download_blob(bucket_name,source_blob_name,file_name)
    except:
        print('file not found. {}'.format(source_blob_name))
        return False
        
    #print('オリジナル画像読み込み完了')
    """
    #----------------------------------------- 画像処理 ------------------------------------------------------------
    img_maker(dir_name)
    """
    #画像データ読み込み
    filename = dir_name + '/2048x1024.jpg'
    try:
        img = cv2.imread(filename)
        height = img.shape[0] # Errorを引っ掛けるためだけの仕掛
    except  AttributeError:
        print('file not found. {}'.format(filename))
        return False

    #オリジナル画像のリサイズ
    height = img.shape[0]
    width = img.shape[1]
    if height != 1024 or width != 2048:
        img = cv2.resize(img , (int(width*(2048.5/width)), int(height*(1024.5/height))))
    #--- ここまでオミット対象 ---
    

    #余白除去後HM読み込み
    filename = dir_name + '/hm_edge.png' # <- dirをつける
    try: 
        img2 = cv2.imread(filename,-1)
        height = img2.shape[0] # Errorを引っ掛けるためだけの仕掛
    except  AttributeError:
        print('file not found.')

    #サイズ調整
    height = img2.shape[0]
    width = img2.shape[1]

    img2_1 = cv2.resize(img2 , (int(width*(2048/width)), int(height*(1024/height))))
    #print('orgサイズ:{}'.format(img.shape))
    #print('リサイズhm:{}'.format(img2_1.shape))
    cv2.imwrite(dir_name + '/resize_heatmap.png', img2_1)
    #print('ヒートマップのサイズ調整完了')

    
    #--- ここから機能オミット ---
    #画像合成
    blended = cv2.addWeighted(src1=img,alpha=0.6,src2=img2_1,beta=0.4,gamma=0.3)

    #作成日時刻印 ---------------------------------------------------------------------------------------------
    str_time = datetime.now(pytz.timezone('Asia/Tokyo')).strftime("%Y-%m-%dT%H:%M:%S")
    cv2.putText(blended, str_time, (10, 30),cv2.FONT_HERSHEY_PLAIN, 1.5,(255, 255, 255), 1, 8)


    #合成後画像保存
    cv2.imwrite(dir_name + '/blended_test.jpg', blended) # <- dirをつける
    #print('合成画像作成完了')

    # 2020/5/13追加
    del img2
    del img2_1
    del blended
    gc.collect()
    #--- ここまで機能オミット ---
    
    
    #半透明画像作成
    #im_rgb = Image.open(dir_name + '/resize_heatmap.png')
    #im_rgb.putalpha(128)
    #im_rgb.save(dir_name + '/pillow_putalpha_solid.png')
    

    #凡例切り出し準備
    im = Image.open(hm_name_sample)

    #凡例切り出し&resize
    #im_crop = im.crop((2230,120,2350,1030))
    im_crop = im.crop((3150,170,3250,1450)) #2020/04/16 凡例切り出しのズレが生じていたので修正
    im_crop_rsize = im_crop.resize((100,1024))   
    #plt.imshow(im_crop_rsize)
    im_crop_rsize.save(dir_name + '/colorbar_crop.jpg', quality=100) # <- dirをつける
    #print('凡例作成完了')

    # 2020/5/13追加
    del im
    del im_crop
    del im_crop_rsize
    gc.collect()
    """
    
    img_blend(dir_name)
    """
    #--- ここから機能オミット ---
    #ブランク画像作成
    height = 1024
    width = 2200
    blank = np.zeros((height, width, 3))
    blank += 255 #←全ゼロデータに255を足してホワイトにする
 
    cv2.imwrite(dir_name + '/blank.jpg',blank) # <- dirをつける
    #print('ベース画像作成完了')


    #凡例付きヒートマップ作成
    img0 = cv2.imread(dir_name + '/blank.jpg') # ブランク画像
    img1 = cv2.imread(dir_name + '/blended_test.jpg') # ヒートマップ合成画像
    img2 = cv2.imread(dir_name + '/colorbar_crop.jpg') # 凡例

    img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    base_img = img0


    #ヒートマップ画像貼り付け基準点設定
    x_offset=0
    y_offset=0

    #ヒートマップ画像合成
    base_img[y_offset:y_offset+img1.shape[0], x_offset:x_offset+img1.shape[1]] = img1

    #凡例画像貼り付け基準点設定
    x_offset=2050
    y_offset=0

    #凡例画像合成
    base_img[y_offset:y_offset+img2.shape[0], x_offset:x_offset+img2.shape[1]] = img2
    base_img = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGB)

    #media_dir = './medium_items/media/' + media_id 
    media_dir = dir_name + '/' + media_id + '.jpg' # 凡例付き合成画像

    #if not os.path.exists(media_dir):
        #os.makedirs(media_dir) #dirが無い場合作成

    #dir_file_name = './medium_items/media/' + media_id + '/heatmap.jpg'

    #cv2.imwrite(dir_file_name,base_img)
    cv2.imwrite(media_dir,base_img)
    #print('凡例付き合成画像作成完了')
    #--- ここまで機能オミット ---
    
    #--- ここから機能オミット ---
    #合成ヒートマップ画像アップローダー
    #- 本番環境設定 -
    #bucket_name = 'rent-production'
    #destination_blob_name = 'reports/heat_maps/hm/media/' + media_id + '/heatmap.jpg' 
    """

    #bucket_name = "heatmap-staging" #<- 将来的には本番環境へ変更
    bucket_name = 'rent-production' #本番環境
    source_file_name = media_dir = dir_name + '/' + media_id + '.jpg'  # 凡例付き合成画像
    #destination_blob_name = 'medium_items/hm/media/' + media_id + '/heatmap.jpg' #テスト環境
    destination_blob_name = 'heatmaps/media/' + media_id + '/heatmap.jpg' #本番環境
    upload_blob(bucket_name, source_file_name, destination_blob_name) #画像データアップロード
    #--- ここまで機能オミット ---

    """
    #半透明ヒートマップ画像アップローダー
    #bucket_name = "rent-production" #"heatmap-staging" #<- 将来的には本番環境へ変更
    bucket_name = "heatmap-staging"
    source_file_name = dir_name + '/pillow_putalpha_solid.png' # 半透明画像
    destination_blob_name = 'medium_items/hm/media/' + media_id + '/pillow_putalpha_solid.png'
    upload_blob(bucket_name, source_file_name, destination_blob_name) #画像データアップロード

    #凡例画像アップローダー
    #bucket_name = "rent-production" #"heatmap-staging" #<- 将来的には本番環境へ変更
    bucket_name = "heatmap-staging"
    source_file_name = dir_name + '/colorbar_crop.jpg' # 凡例画像
    destination_blob_name = 'medium_items/hm/media/' + media_id + '/colorbar_crop.jpg'
    upload_blob(bucket_name, source_file_name, destination_blob_name) #画像データアップロード
    """
    #作成画像削除
    os.remove(dir_name + "/hm_org.jpg")
    os.remove(dir_name + "/hm.png")
    os.remove(dir_name + '/hm_edge.png')
    os.remove(dir_name + '/2048x1024.jpg') # オミット対象
    os.remove(dir_name + '/blended_test.jpg') # オミット対象 
    os.remove(dir_name + '/resize_heatmap.png')
    #os.remove(dir_name + '/pillow_putalpha_solid.png')
    os.remove(dir_name + '/colorbar_crop.jpg') 
    os.remove(dir_name + '/blank.jpg') # オミット対象
    #os.remove(media_dir) # オミット対象
    

    #return True    
    
#GCSへのデータアップローダー    
def upload_blob(bucket_name, source_file_name, destination_blob_name):
        #Uploads a file to the bucket.
        # bucket_name = "your-bucket-name"
        # source_file_name = "local/path/to/file"
        # destination_blob_name = "storage-object-name"

        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)

        blob.upload_from_filename(source_file_name)
        """
        print(
            "File {} uploaded to {}.".format(
                source_file_name, destination_blob_name
            )
        )
        """

def down_blob(bucket_name,source_blob_name,file_name):
        #------------------------------------- GCSからの画像データ読み込み -------------------------------------------------
    #直接画像読み込み
    #os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/tani_kyuichiro/nurve-cloud-98b3f-bb7c97f8fb03.json'
    
    
    #ファイルの存在確認、拡張子取得
    bucketname = 'rent-production'
    prefix = 'medium_items/media/'+media_id + '/'
    ext = extension_get(bucketname,prefix) #拡張子吸い出し
    if not ext:
        return False
    
    #bucket_name = 'rent-production'
    #source_blob_name = 'medium_items/media/'+media_id + '/2048x1024.' + ext
    #file_name = './tmp/2048x1024.jpg'
    try:
        download_blob(bucket_name,source_blob_name,file_name)
    except:
        print('file not found. {}'.format(source_blob_name))
        return False
        
    #print('オリジナル画像読み込み完了')
    
#バケット内のオブジェクトリスト取得
def list_blobs(bucket_name):
        """Lists all the blobs in the bucket."""
        # bucket_name = "your-bucket-name"

        storage_client = storage.Client()

        # Note: Client.list_blobs requires at least package version 1.17.0.
        blobs = storage_client.list_blobs(bucket_name)
        
        blobs_lst = []
        for blob in blobs:
            #print(blob.name)
            blobs_lst.append(blob.name)
        return blobs_lst    

def loop_check(data):
        print("data['name']:{}".format(data['name']))
        if re.match(r'medium_items/media/[0-9]+/heatmap\.json',data['name']):
          return True  

#------------------------------------ メソッド -----------------------------------------

#def main(data):
def heatmap_maker2(request):
    """#強制終了
    if not loop_check(data):
        sys.exit(1)"""
    #media_id受取
    #print('受信開始')
    request_json = request.get_json(silent=True)
    request_args = request.args

    if request_json and 'data' in request_json:
        media_ids = request_json['data']
        cnt = request_json['cnt']
        lcnt = request_json['lcnt']
    elif request_args and 'data' in request_args:
        media_ids = request_args['data']
        cnt = request_json['cnt']
        lcnt = request_json['lcnt']
    else:
        media_ids = False
        return escape(media_ids)  

    #ディレクトリ準備
    #print('Dir作成')
    tmpdir = tempfile.TemporaryDirectory()
    dir_name = tmpdir.name
    """
    #media_idのファイル名リスト作成
    blobs_lst = list_blobs('heatmap-staging')
    json_lst = []
    for blob in blobs_lst:
        if re.search(r'medium_items/media/[0-9]+/heatmap\.json',blob):
            json_lst.append(blob)
    """

    #blobからjsonファイル取得してヒートマップ作成
    #for json in json_lst:
    #json = data['name']
    #num = re.search(r'\d+',json)
    #media_id = num.group()
    #bucket_name = 'rent-production' #'heatmap-staging'
    #bucket_name = 'heatmap-staging'
    bucket_name = 'rent-production'
    #source_blob_name = 'reports/heat_maps/media/' + media_id + '/coordinate.json' #'medium_items/media/' +media_id + '/heatmap.json'
    
    #------- debug -------
    test_lst = [14687738, 16287716, 13809236, 13808540, 13809014, 16376628, 14688089] 
    #------- debug -------  
    llcnt = 1
    for media_id in media_ids:
        if media_id == '':
            continue
        #debug
        if media_id in test_lst:
            print('----- Hit!!---- {}'.format(media_id))
        #debug
        
        #source_blob_name = 'medium_items/media/' +str(media_id) + '/heatmap.json'
        source_blob_name = 'heatmaps/media/' + str(media_id) + '/coordinate.json'
        #source_blob_name = json
        file_name = dir_name + '/' + str(media_id) + '.json'
        #blobからjsonファイル取得
        #print('json取得開始')
        download_blob(bucket_name,source_blob_name,file_name)
    
        #ヒートマップ作成
        #print('ヒートマップ作成開始')
        #print('m_id:{}'.format(media_id))
        heatmap_maker_json(file_name,dir_name)
    
        #os.remove(file_name) #json削除 #2020/5/13 機能停止
        
        #print('cnt:{}/lcnt:{}/llcnt:{}'.format(cnt,lcnt,llcnt))
        
        if len(media_ids) <= llcnt:
            print('cnt:{}/lcnt:{}/llcnt:{}/mcnt:{}'.format(cnt,lcnt,llcnt,len(media_ids)))
            if len(media_ids) < 60:
                print('--cnt:{}/lcnt:{}/llcnt:{}/mcnt:{}'.format(cnt,lcnt,llcnt,len(media_ids)))
            return escape(media_id)
        
        time.sleep(1)
        llcnt += 1

    tmpdir.cleanup() #仮想dir削除

    #return escape(media_ids)


In [61]:
json_lst = ["10011419"]

In [62]:
#ディレクトリ準備
#print('Dir作成')
tmpdir = tempfile.TemporaryDirectory()
dir_name = tmpdir.name


#blobからjsonファイル取得してヒートマップ作成
for json in json_lst:
    num = re.search(r'\d+',json)
    media_id = num.group()
    bucket_name = 'rent-production'
    source_blob_name = 'heatmaps/media/'+media_id + '/coordinate.json'
    file_name = dir_name + '/' + str(media_id) + '.json'
    #blobからjsonファイル取得
    download_blob(bucket_name,source_blob_name,file_name)
    
    #ヒートマップ作成
    heatmap_maker_json(file_name,dir_name)
    
    #os.remove(file_name)

tmpdir.cleanup() #仮想dir削除
    

lst_2d: [[0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, 1, 0, 1, 2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0], [0, 3, 1, 2, 1, 0, 1, 0, 0, 2, 1, 2, 1, 2, 0, 0, 0, 0, 1, 2], [3, 4, 5, 3, 2, 4, 5, 1, 3, 4, 2, 0, 1, 2, 7, 7, 4, 4, 6, 4], [1, 3, 6, 1, 2, 2, 4, 1, 3, 0, 2, 1, 2, 3, 3, 1, 4, 4, 2, 1], [3, 3, 1, 2, 2, 2, 0, 0, 1, 0, 1, 0, 0, 1, 1, 5, 3, 5, 2, 2], [1, 2, 2, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 2, 1, 1, 3, 1], [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],

# csv2json実験環境

In [9]:
#ライブラリ
import os
import sys
from google.cloud import storage as gcs
import pandas as pd
from io import BytesIO

#from google.cloud import storage

import re

import json
import tempfile
import gc

import asyncio
import requests
from requests.exceptions import Timeout

from flask import escape
import time
import pprint

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/tani_kyuichiro/nurve-cloud-98b3f-bb7c97f8fb03.json'

def download_blob(bucket_name,file_name):
    # create gcs cliaent
    PROJECT_NAME = "nurve-cloud-98b3f"
    client = gcs.Client(PROJECT_NAME)
    bucket = client.get_bucket(bucket_name)
    # create blob
    blob = gcs.Blob(file_name, bucket)
    content = blob.download_as_string()
    return pd.read_csv(BytesIO(content))

#データダウンロード
print('ファイルダウンロード開始')
bucket_name = "heatmap-production-csv"
file_name = "logging/heatmap.csv"
project_name = "nurve-cloud-98b3f"
df = download_blob(bucket_name,file_name)
#データ処理
print('データ処理開始')

media_id_lst = df['media_id'].values.tolist()
unique_media_id_lst = list(set(media_id_lst))

ファイルダウンロード開始
データ処理開始


In [10]:
df

,media_id,x,y
0,5254,0.0,209.2
1,5254,4.2,266.0
2,5254,4.2,239.0
3,5254,6.2,288.8
4,5254,10.0,320.4
...,...,...,...
20355167,18516491,31.0,256.6
20355168,18516491,85.8,341.6
20355169,18516491,28.0,38.2
20355170,18516491,-20.8,50.6


In [6]:
mid = [9830418, 14680084, 2621465, 9830436, 12714058, 16646221, 12714072, 12714074, 17432693, 17432694, 17432695, 17432698, 17432699, 17432700, 10354821, 6684816, 6684824, 14942360, 6684830, 6684835, 9830568, 15335593, 16777386, 655531, 16777406, 17694928, 3801300, 17694935, 3801304, 10748130, 10748131, 4456678, 4456685, 3145984, 15335694, 15335697, 3146012, 17480665, 5898558, 14680387, 10355019, 14680395, 17432922, 16646512, 16646513, 16646516, 6029686, 6029689, 16646521, 16646526]

In [7]:
df = df[df['media_id'].isin(mid)].reset_index(drop=True)

In [8]:
df

,media_id,x,y
0,655531,0.0,359.9
1,655531,2.6,358.5
2,655531,-13.8,356.1
3,655531,0.0,359.9
4,655531,-25.4,359.5
...,...,...,...
15497,17694935,-29.0,228.6
15498,17694935,-51.6,221.4
15499,17694935,26.4,253.2
15500,17694935,-2.2,239.4


In [3]:
import os
import sys
from google.cloud import storage as gcs
import pandas as pd
from io import BytesIO

#from google.cloud import storage

import re

import json
import tempfile
import gc

import asyncio
import requests
from requests.exceptions import Timeout

from flask import escape
import time
import pprint

#GCSへのデータアップローダー    
def upload_blob(bucket_name, source_file_name, destination_blob_name):
        """Uploads a file to the bucket."""
        # bucket_name = "your-bucket-name"
        # source_file_name = "local/path/to/file"
        # destination_blob_name = "storage-object-name"

        #storage_client = storage.Client()
        storage_client = gcs.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)

        blob.upload_from_filename(source_file_name)
        """
        print(
            "File {} uploaded to {}.".format(
                source_file_name, destination_blob_name
            )
        )
        """
def download_blob(bucket_name,file_name):
    # create gcs cliaent
    PROJECT_NAME = "nurve-cloud-98b3f"
    client = gcs.Client(PROJECT_NAME)
    bucket = client.get_bucket(bucket_name)
    # create blob
    blob = gcs.Blob(file_name, bucket)
    content = blob.download_as_string()
    return pd.read_csv(BytesIO(content))
"""
def download_blob(bucket_name, source_blob_name, destination_file_name):
    #Downloads a blob from the bucket.
    # bucket_name = "your-bucket-name"
    # source_blob_name = "storage-object-name"
    # destination_file_name = "local/path/to/file"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(
        "Blob {} downloaded to {}.".format(
            source_blob_name, destination_file_name
        )
    )
"""

#----------------------- 非同期 ------------------------

async def req_url(cnt,lcnt,*media_id):
    print('非同期開始')
    loop = asyncio.get_event_loop()
    
    try: 
        response = requests.put(
                'https://asia-northeast1-nurve-cloud-98b3f.cloudfunctions.net/heatmap_maker',
                json.dumps({'data' : media_id, "cnt" : cnt, "lcnt" : lcnt}),
                headers={'Content-Type': 'application/json'}, timeout=8)
        print('res_code:{}'.format(response.status_code))
        #loop.run_in_executor(None, response)        
    except Timeout:
            #print('{} continued'.format(media_id))
            print('continued')

async def main_loop(cnt,lcnt,*media_id):
    #loop = asyncio.get_event_loop()
    
    #loop.run_until_complete(asyncio.gather(*[req_url(x) for x in media_id_lst]))
    await req_url(cnt,lcnt,*media_id)


def nom_req_url(*media_id):
    #print('同期開始')
    
    try: 
        response = requests.put(
                'https://asia-northeast1-nurve-cloud-98b3f.cloudfunctions.net/heatmap_maker',
                json.dumps({'data' : media_id}),
                headers={'Content-Type': 'application/json'})#, timeout=0.8)
        print(response)        
    except Timeout:
            print('{} continued'.format(media_id))

def json_maker(media_id,*xy_lst):
    tmp_dic1 = {}
    tmp_lst = []

    for xy in xy_lst:
        tmp_lst.append(xy)

    tmp_dic1['coordinate'] = tmp_lst
    
    #ディレクトリ準備
    tmpdir = tempfile.TemporaryDirectory()
    dir_name = tmpdir.name

    #json保存
    savepath = dir_name + '/' + media_id + '.json'
    #savepath = media_id + '.json'
    with open(savepath, 'w') as outfile:
        json.dump(tmp_dic1, outfile)
    
    #GCSへアップロード
    bucket_name = 'rent-production'
    #bucket_name = 'heatmap-staging'
    source_file_name = savepath
    destination_blob_name = 'heatmaps/media/' + media_id + '/coordinate.json'
    #destination_blob_name = 'medium_items/media/' +media_id + '/heatmap.json'
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    #print('upload完了')
    os.remove(source_file_name) #ローカルで作成したjsonファイル削除
    #ディレクトリ解除
    tmpdir.cleanup()

def chunk_maker(CHNK,*media_ids):
    chunks = []
    lst_cnt = len(media_ids)
    print(lst_cnt)
    media_ids = list(media_ids)
    print(type(media_ids))

    for i in range(0,lst_cnt,CHNK):
        #if len(unique_media_id_lst) >= CHNK + 1:
        ch = media_ids[0:CHNK+1]
        ch1 = [a for a in ch if a != '']
        if len(ch1) >= 1:
            #ch1 = [a for a in ch if a != '']
            chunks.append(ch1)
            del media_ids[0:CHNK+1]
            #print('chunks:{}'.format(chunks))

        #loop.run_until_complete(asyncio.gather(req_url(m)))
        #print('cnt:{}'.format(cnt))
    return chunks

In [4]:
def csv2json(*request):
    """
    #media_id受取
    request_json = request.get_json(silent=True)
    request_args = request.args

    if request_json and 'data' in request_json:
        media_ids = request_json['data']
        cnt = request_json['cnt']
    elif request_args and 'data' in request_args:
        media_ids = request_args['data']
        cnt = request_json['cnt']
    else:
        media_ids = False
    """

    media_ids = list(request)
    """
    #データダウンロード
    #print('ファイルダウンロード開始')
    bucket_name = "heatmap-production-csv"
    file_name = "logging/heatmap.csv"

    df = download_blob(bucket_name,file_name)
    """
    """
    #データ処理
    loop = asyncio.get_event_loop()
    #print('データ処理開始')
    #debug
    #cnt = 1
    for m in media_ids:

        tmp_dic1 = {}
        tmp_lst = []
    
        #データ選別
        df_tmp = df[df['media_id'] == m ].reset_index(drop=True)
    
        #del df #dfの削除
        #gc.collect()    
    
        media_id = str(int(df_tmp.iloc[0]['media_id']))

        #x,y座標データ取得
        xy_lst = df_tmp[['x','y']].values.tolist()

        #新規改変json_uploading
        json_maker(media_id,*xy_lst)

        del df_tmp #dfの削除
        gc.collect() 
        #print("media_id_lst : {}".format(media_ids))

    del df #dfの削除
    gc.collect() 
    """
    #n個のmedia_idをlistにして固める
    CHNK =  49 #499
    print('len:{}'.format(len(media_ids)))
    chunks = chunk_maker(CHNK,*media_ids)
    
    #chunks = []
    #lst_cnt = len(media_ids)
    #print(lst_cnt)
    
    """for i in range(0,lst_cnt,CHNK):
        ch = media_ids[0:CHNK+1]
        ch1 = [a for a in ch if a != '']
        if len(ch1) >= 1:
            #ch1 = [a for a in ch if a != '']
            chunks.append(ch1)
            del media_ids[0:CHNK+1]
            #print('chunks:{}'.format(chunks))"""

        #loop.run_until_complete(asyncio.gather(req_url(m)))
        #print('cnt:{}'.format(cnt))
    
    cnt = 0
    lcnt = 1
    for m in chunks:
        loop.run_until_complete(asyncio.gather(req_url(cnt,lcnt,*m)))
        #loop.run_until_complete(asyncio.gather(main_loop(cnt,local_cnt,*m)))
        #nom_req_url(*m)
    lcnt += 1
    
        #if local_cnt >= len(chunks):
            #sys.exit(1)
        #time.sleep(2)
        #print(m)

In [59]:
csv2json(*unique_media_id_lst)

len:46416
46416
<class 'list'>
[9830418, 14680084, 2621465, 9830436, 12714058, 16646221, 12714072, 12714074, 17432693, 17432694, 17432695, 17432698, 17432699, 17432700, 10354821, 6684816, 6684824, 14942360, 6684830, 6684835, 9830568, 15335593, 16777386, 655531, 16777406, 17694928, 3801300, 17694935, 3801304, 10748130, 10748131, 4456678, 4456685, 3145984, 15335694, 15335697, 3146012, 17480665, 5898558, 14680387, 10355019, 14680395, 17432922, 16646512, 16646513, 16646516, 6029686, 6029689, 16646521, 16646526]
[6029696, 10748289, 10748290, 17564034, 10748293, 10748294, 10748295, 10748296, 6029705, 10748299, 10748301, 10748302, 6029713, 10748307, 10748310, 16777634, 16646570, 16646573, 16646574, 16646577, 16646578, 16646581, 16646582, 17564094, 15335883, 14680534, 3670487, 17564160, 17302020, 4063749, 17302021, 17302022, 17302023, 15073801, 17302025, 17302027, 15073811, 17302035, 17302036, 11534873, 10355226, 10355230, 10355238, 11797040, 10355251, 14680629, 14680632, 15336003, 15336005, 6

In [46]:
unique_media_id_lst

[]

In [24]:
df_s

,9880851
coordinate,"[[0.0, 14.000000000003185], [-1.20000000000004..."
item_id,4397602
organization_group_id,1


In [ ]:
#----------------------------------------- GCSへのアップロード -----------------------------------------------
    # Create a storage client.

    #os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/tani_kyuichiro/nurve-cloud-98b3f-bb7c97f8fb03.json'

    storage_client = google.cloud.storage.Client()

    bucket_name = 'heatmap-staging' 
    bucket = storage_client.get_bucket(bucket_name)

    source_file_name = './data/' + str(media_id) + '.json'
    destination_blob_name = 'medium_items/media/' + str(media_id) + '/heatmap.json'
    blob = bucket.blob(destination_blob_name)
 
    # Upload the local file to Cloud Storage.
    blob.upload_from_filename(source_file_name)
 
    print('File {} uploaded to {}.'.format(source_file_name,bucket))
    print('GCSへのアップロード完了') 
    
    #---------------------------------------------- ディレクトリ削除 ----------------------------------------------------------------
    #shutil.rmtree('./medium_items/media/' + media_id)

In [ ]:
df_sort = df.sort_values(['organization_group_id','media_id'])

In [ ]:
df_sort

In [ ]:
#item_idリスト作成
df_s = df['item_id']
#item_s = df_s('item_id')
#item_lst = item_s.values.tolist()
item_lst = df_s.values.tolist()
item_lst = list(set(item_lst))

In [ ]:
for id in item_lst:
    lines = df2[df2['item_id']==id].values.tolist()
    if len(lines) > 100:
        print(len(lines))
        if not heatmap_maker2(lines):
            print('---- error ----')
            continue

In [ ]:
#いずれはコマンド入力へ変更
val = 'item_id'
id = '7031202' #item_id
#val = 'media_id'
#file = '15122174'
#media_id = '13106716'

In [7]:
# -*- coding:utf-8 -*-
#ライブラリ
import os
from google.cloud import storage as gcs
import pandas as pd
from io import BytesIO

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/tani_kyuichiro/nurve-cloud-98b3f-bb7c97f8fb03.json'

bucket_name = "heatmap-staging"
file_name = "logging/heatmap.csv"
project_name = "nurve-cloud-98b3f"

# create gcs cliaent
client = gcs.Client(project_name)
bucket = client.get_bucket(bucket_name)
# create blob
blob = gcs.Blob(file_name, bucket)
content = blob.download_as_string()
train = pd.read_csv(BytesIO(content))
train.head()

,organization_group_id,media_id,item_id,x,y
0,1,365,178,-13.2,273.4
1,1,365,178,-18.2,316.4
2,1,365,178,-13.2,273.4
3,1,365,178,-19.4,331.6
4,1,366,179,0.0,122.0


In [ ]:
heatmap_maker(val,id)

In [ ]:
#GCSへのアップロード
# Create a storage client.

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/tani_kyuichiro/nurve-cloud-98b3f-bb7c97f8fb03.json'

storage_client = google.cloud.storage.Client()

bucket_name = 'rent-heatmap' 
bucket = storage_client.get_bucket(bucket_name)

source_file_name = 'medium_items/media/' + media_id + '/heatmap.jpg'
blob = bucket.blob(source_file_name)
 
# Upload the local file to Cloud Storage.
blob.upload_from_filename(source_file_name)
 
print('File {} uploaded to {}.'.format(source_file_name,bucket))
print('GCSへのアップロード完了')

In [ ]:
import datetime

dt_now = datetime.datetime.now()
today = dt_now.strftime("%Y-%m-%d")

In [ ]:
num = '6421364'
sql1 = "SELECT id as media_id, item_id ,x ,y FROM panorama_user_event t1 \
        JOIN ( SELECT id, mediumable_id FROM nurvecloud.media WHERE mediumable_type = 'Item' ) t2 \
        ON mediumable_id = item_id \
        WHERE item_id = "
sql2 = " AND x IS NOT NULL AND y IS NOT NULL  AND \
        TD_TIME_RANGE(t1.time, TD_TIME_ADD('"
sql3 = today + "','-7d','JST'),'"
sql4 = today + "', 'JST')"
sql = sql1 + num + sql2 + sql3 + sql4
print(sql)

In [ ]:
engine = td.create_engine('presto:rails_events_production')

# Read Treasure Data query into a DataFrame.
df = td.read_td(sql, engine)
lines = df.values.tolist()

In [ ]:
#一気にデータを吸い取るsql
sql1 = "WITH item_id_list as (SELECT id as media_id,t1.item_id,mediumable_id,cnt \
  ,TD_TIME_FORMAT(resent_time,'yyyy-MM-dd HH:mm:ss','JST') as jp_time \
FROM \
  (SELECT item_id,COUNT(*) as cnt FROM panorama_user_event \
  WHERE TD_TIME_RANGE(time, TD_TIME_ADD('" + today + "','-8d','JST'), '" + today + "', 'JST') \
  GROUP BY item_id \
  ORDER BY cnt DESC ) t1 \
JOIN \
  (SELECT id, mediumable_id FROM nurvecloud.media WHERE mediumable_type = 'Item') t2 ON mediumable_id = item_id \
JOIN \
  (SELECT item_id ,MAX(time) OVER(PARTITION BY item_id) as resent_time FROM panorama_user_event WHERE \
    TD_TIME_RANGE(time, TD_TIME_ADD('" + today + "','-8d','JST'), '" + today + "', 'JST')) t3 \
ON t1.item_id = t3.item_id ORDER BY resent_time DESC, cnt DESC) \
, item_id_selecter as (SELECT DISTINCT * FROM item_id_list WHERE cnt > 100)"

sql2 = "SELECT media_id,t1.item_id ,x ,y FROM panorama_user_event t1 join item_id_selecter t2 on t1.item_id = t2.item_id \
where  x IS NOT NULL AND y IS NOT NULL  AND \
TD_TIME_RANGE(time, TD_TIME_ADD('" + today + "','-8d','JST'), '" + today + "', 'JST')"

sql = sql1 + sql2

In [ ]:
engine = td.create_engine('presto:rails_events_production')

# Read Treasure Data query into a DataFrame.
df2 = td.read_td(sql, engine)
#lines2 = df2.values.tolist()

In [ ]:
#item_idリスト作成
df_s = df2['item_id']
#item_s = df_s('item_id')
#item_lst = item_s.values.tolist()
item_lst = df_s.values.tolist()
item_lst = list(set(item_lst))


In [ ]:
item_lst.sort()
for id in item_lst:
    lines = df2[df2['item_id']==id].values.tolist()
    if len(lines) > 100:
        print(len(lines))
        if not heatmap_maker2(lines):
            print('---- error ----')
            continue
    

In [ ]:
lines = df2[df2['item_id']==7225719].values.tolist()
print(type(lines))

In [ ]:
#line = lines[0]
#str(int(line[1]))
heatmap_maker2(lines)

In [ ]:
bucket_name = 'rent-production'
list_blobs(bucket_name)

In [ ]:
client = storage.Clinet()
bucket = client.get_bucket('rent-production')
blobs = bucket.list_blobs(prefix="medium_items/media/", delimiter="/")
dirs = []
for page in blobs.pages:
    dirs.extend(page.prefixes)

[ print(x) for x in dirs ]

In [ ]:
def extension_get(bucket_name,prefix):
    storage_client = storage.Client()

        # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(
            bucket_name, prefix=prefix, delimiter='/'
        )

    blob_lst = []
    target = prefix

    for blob in blobs:
        blob_lst.append(blob.name)
        print(blob.name)
    l_in = [s for s in blob_lst if target in s]

    if len(l_in) > 0:
        extension = l_in[0].replace(target,'')
        print('ext: {}'.format(extension))
        return extension
    else:
        return False

In [ ]:
    bucketname = 'rent-production'
    prefix = 'medium_items/media/'
    ext = extension_get(bucketname,prefix) #拡張子吸い出し